In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from typing import Optional

DATASETS_DIR = "data"

class GPUDataset(Dataset):
    def __init__(self, dataset: Dataset, transform: Optional[transforms.Compose], device: str):
        self.device = device
        self.transform = transform

        self.data = torch.empty((len(dataset), *transform(dataset[0][0]).shape), device=device)
        self.targets = torch.empty(len(dataset), dtype=torch.long, device=device)

        for i, (img, target) in enumerate(dataset):
            self.data[i] = transform(img).to(device)
            self.targets[i] = torch.tensor(target).to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


def get_dataset(
    dataset_name: str = "FashionMNIST", transform: Optional[transforms.Compose] = None
) -> tuple[Dataset, Dataset]:
    if dataset_name == "FashionMNIST":
        train_dataset = datasets.FashionMNIST(root=DATASETS_DIR, train=True, download=True)
        test_dataset = datasets.FashionMNIST(root=DATASETS_DIR, train=False, download=True)
    elif dataset_name == "CIFAR100":
        train_dataset = datasets.CIFAR100(
            root=DATASETS_DIR, train=True, download=True, transform=transform
        )
        test_dataset = datasets.CIFAR100(
            root=DATASETS_DIR, train=False, download=True, transform=transform
        )

    return train_dataset, test_dataset


In [ ]:
import torch
from torchvision import transforms
from torchvision.models import MobileNet_V3_Small_Weights
from torch.utils.data import DataLoader, Dataset, Subset, random_split
import numpy as np
from typing import Optional
import os


def get_dataloaders(
    dataset_name: str = "FashionMNIST",
    batch_size: int = 32,
    val_split: float = 0.2,
    seed: int = 42,
    num_workers: int = 0,
    transform: Optional[transforms.Compose] = None,
    device: str = "cuda",
) -> tuple[Dataset, DataLoader, DataLoader, DataLoader]:

    torch.manual_seed(seed)

    if dataset_name == "FashionMNIST":
        train_dataset, test_dataset = get_dataset(dataset_name=dataset_name)
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )
    elif dataset_name == "CIFAR100":
        train_dataset, test_dataset = get_dataset(dataset_name=dataset_name, transform=transform)
        weights = MobileNet_V3_Small_Weights.DEFAULT
        transform = weights.transforms()
    else:
        raise ValueError(f"Unknown dataset name: {dataset_name}")

    val_size = int(val_split * len(train_dataset))
    train_size = len(train_dataset) - val_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    if dataset_name == "FashionMNIST":
        train_dataset = GPUDataset(train_dataset, transform, device)
        val_dataset = GPUDataset(val_dataset, transform, device)
        test_dataset = GPUDataset(test_dataset, transform, device)

    train_dataloader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    val_dataloader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    test_dataloader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )

    return train_dataset, train_dataloader, val_dataloader, test_dataloader


def get_subset_loader(
    dataset: Dataset, mask: np.ndarray, batch_size: int = 32, num_workers: int = 0
) -> DataLoader:
    assert mask.dtype == bool, "Mask must be a boolean array."
    assert len(mask) == len(dataset), "Mask and dataset must have the same length."
    indices = np.where(mask)[0]
    subset = Subset(dataset, indices)
    return DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=num_workers)



num_workers = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_name = "FashionMNIST"
train_dataset, train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    dataset_name=dataset_name,
    num_workers=num_workers,
    device=device,
)

total_samples = len(train_dataset)
mask = np.zeros(total_samples, dtype=bool)
mask[:500] = True

subset_loader = get_subset_loader(train_dataset, mask, num_workers=num_workers)

print(f"Train dataloader: {len(train_dataloader)}")
print(f"Validation dataloader: {len(val_dataloader)}")
print(f"Test dataloader: {len(test_dataloader)}")
print(f"Subset dataloader: {len(subset_loader)}")

if dataset_name == "FashionMNIST":
    print(subset_loader.dataset.dataset.data.device)
    print(subset_loader.dataset.dataset.targets.device)


100%|██████████| 26.4M/26.4M [00:01<00:00, 14.3MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 114kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 4.24MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 4.50MB/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Train dataloader: 1500
Validation dataloader: 375
Test dataloader: 313
Subset dataloader: 16
cpu
cpu


In [ ]:
from torch import optim
from tqdm import tqdm
import torch.nn as nn

def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    num_epochs: int = 2,
    learning_rate: float = 1e-3,
):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0
        for images, labels in train_loader:

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()


def validate_model(model: nn.Module, val_loader: DataLoader):

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    criterion = nn.CrossEntropyLoss()

    with torch.inference_mode():
        for images, labels in val_loader:
            output = model(images)
            loss = criterion(output, labels)
            val_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    correct
    return val_loss, accuracy

In [ ]:
import torch.nn.functional as F

class SimpleFCN(nn.Module):
    def __init__(self):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = SimpleFCN()

In [ ]:
percentage = range(5, 101, 5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in percentage:
    result = 0
    acc = 0
    for j in range(10):
        subsample = int(len(train_dataset) * i / 100)
        mask = np.zeros(len(train_dataset), dtype=bool)
        random_indices = np.random.choice(len(train_dataset), subsample, replace=False)
        mask[random_indices] = True

        subset_loader = get_subset_loader(train_dataset, mask, num_workers=num_workers)
        model = SimpleFCN().to(device)
        train_model(model, subset_loader)
        loss1, acc1 = validate_model(model, val_dataloader)
        result += loss1
        acc += acc1
    print(f"Percentage: {i} , loss={result/10} acc={acc/10}")



100%|██████████| 2/2 [00:00<00:00,  5.90it/s]


Percentage: 5 , loss=234.2657291650772 acc=0.7690333333333332


100%|██████████| 2/2 [00:00<00:00,  3.03it/s]


Percentage: 10 , loss=214.3893195167184 acc=0.7912416666666667


100%|██████████| 2/2 [00:00<00:00,  2.03it/s]


...

Percentage: 90 , loss=147.3020055092871 acc=0.8577666666666668


100%|██████████| 2/2 [00:07<00:00,  3.78s/it]


Percentage: 95 , loss=146.0119609594345 acc=0.8582583333333333


100%|██████████| 2/2 [00:07<00:00,  3.82s/it]


Percentage: 100 , loss=144.7901761163026 acc=0.8595416666666666


In [ ]:
model = SimpleFCN().to(device)
subsample = int(len(train_dataset) * 1 / 100)
mask = np.zeros(len(train_dataset), dtype=bool)
random_indices = np.random.choice(len(train_dataset), subsample, replace=False)
mask[random_indices] = True

subset_loader = get_subset_loader(train_dataset, mask, num_workers=num_workers)
train_model(model, subset_loader)
loss, acc = validate_model(model, val_dataloader)
print(f"loss={loss} acc={acc}")



100%|██████████| 2/2 [00:00<00:00, 23.88it/s]


loss=346.98670893907547 acc=0.6820833333333334
